In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

2023-11-01 09:09:43.126281: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-01 09:09:43.891439: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-01 09:09:43.891521: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-01 09:09:43.895586: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-01 09:09:44.316857: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-01 09:09:44.319801: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [3]:
filenames=os.listdir("train")

categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)

df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

In [4]:
df

,filename,category
0,dog.3167.jpg,1
1,cat.4508.jpg,0
2,cat.8644.jpg,0
3,dog.4302.jpg,1
4,dog.1426.jpg,1
...,...,...
15432,dog.25.jpg,1
15433,cat.5690.jpg,0
15434,dog.5408.jpg,1
15435,dog.4655.jpg,1


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D, \
    Dropout,Flatten,Dense,Activation, \
    BatchNormalization
# from keras.preprocessing.image import load_img
from PIL import Image
import PIL

model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',metrics=['accuracy'])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 126, 126, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 61, 61, 64)        2

In [7]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [8]:
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.20,
                                        random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15

In [10]:
train_df

,filename,category
0,cat.5626.jpg,cat
1,cat.7208.jpg,cat
2,cat.9433.jpg,cat
3,cat.8887.jpg,cat
4,dog.4571.jpg,dog
...,...,...
12344,dog.903.jpg,dog
12345,cat.4326.jpg,cat
12346,cat.4761.jpg,cat
12347,cat.1248.jpg,cat


In [11]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                   rescale=1./255,
                                   shear_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1
                                   )

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    "train/",x_col='filename',y_col='category',
                                                    target_size=Image_Size,
                                                    class_mode='categorical',
                                                    batch_size=batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df,
    "train/",
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

test_datagen = ImageDataGenerator(rotation_range=15,
                                  rescale=1./255,
                                  shear_range=0.1,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1)

test_generator = train_datagen.flow_from_dataframe(train_df,
                                                   "test1/",x_col='filename',y_col='category',
                                                   target_size=Image_Size,
                                                   class_mode='categorical',
                                                   batch_size=batch_size)

Found 12349 validated image filenames belonging to 2 classes.
Found 3088 validated image filenames belonging to 2 classes.
Found 0 validated image filenames belonging to 0 classes.


/home/ravshanjon/miniconda3/envs/praktikum/lib/python3.11/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 12349 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [30]:
validate_df

,filename,category
0,cat.8383.jpg,cat
1,dog.3324.jpg,dog
2,cat.9360.jpg,cat
3,cat.1786.jpg,cat
4,dog.4003.jpg,dog
...,...,...
3083,dog.2047.jpg,dog
3084,dog.2627.jpg,dog
3085,cat.2143.jpg,cat
3086,cat.4892.jpg,cat


In [36]:
epochs=10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

Epoch 1/10
823/823 [==============================] - 366s 445ms/step - loss: 0.3967 - accuracy: 0.8277 - val_loss: 0.5668 - val_accuracy: 0.7691 - lr: 0.0010
Epoch 2/10
823/823 [==============================] - 356s 433ms/step - loss: 0.3860 - accuracy: 0.8275 - val_loss: 1.2439 - val_accuracy: 0.7385 - lr: 0.0010
Epoch 3/10
823/823 [==============================] - 354s 430ms/step - loss: 0.3797 - accuracy: 0.8329 - val_loss: 0.3022 - val_accuracy: 0.8686 - lr: 0.0010
Epoch 4/10
823/823 [==============================] - 364s 442ms/step - loss: 0.3723 - accuracy: 0.8369 - val_loss: 0.3452 - val_accuracy: 0.8654 - lr: 0.0010
Epoch 5/10
823/823 [==============================] - 310s 376ms/step - loss: 0.3610 - accuracy: 0.8425 - val_loss: 0.4193 - val_accuracy: 0.8530 - lr: 0.0010
Epoch 6/10
823/823 [==============================] - 346s 420ms/step - loss: 0.3608 - accuracy: 0.8478 - val_loss: 0.4590 - val_accuracy: 0.8553 - lr: 0.0010
Epoch 7/10
823/823 [==========================

In [14]:
model.save("cat-dog-predicter.keras")

In [34]:
test_filenames = os.listdir("test1")
test_df = pd.DataFrame({
    'filename': test_filenames,
    "category":"any_values"
})
nb_samples = test_df.shape[0]